# 1. Importing Required Libraries

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import Callback, LearningRateScheduler
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import r2_score
from tabulate import tabulate

# 2. Custom Accuracy Metric

In [3]:
class R2Score(tf.keras.metrics.Metric):
    def __init__(self, name="r2_score", **kwargs):
        super().__init__(name=name, **kwargs)
        self.sse = self.add_weight(name="sse", initializer="zeros")
        self.sst = self.add_weight(name="sst", initializer="zeros")
        self.count = self.add_weight(name="count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, self.dtype)
        y_pred = tf.cast(y_pred, self.dtype)
        mean_y = tf.reduce_mean(y_true)

        self.sse.assign_add(tf.reduce_sum(tf.square(y_true - y_pred)))
        self.sst.assign_add(tf.reduce_sum(tf.square(y_true - mean_y)))
        self.count.assign_add(tf.cast(tf.size(y_true), self.dtype))

    def result(self):
        return 1.0 - (self.sse / (self.sst + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.sse.assign(0.0)
        self.sst.assign(0.0)
        self.count.assign(0.0)

# 3. Custom Epoch Logger

In [4]:
class EpochLogger(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"\n📌 Epoch {epoch+1:03d}\n")
        headers = ["Output", "MSE", "MAE", "R²"]
        rows = []
        output_keys = ['Collapse_factor', 'Hinge_1', 'Hinge_2']
        for prefix in ["", "val_"]:
            for out in output_keys:
                row = [f"{prefix}{out}"]
                for metric in ['mse', 'mae', 'r2_score']:
                    key = f"{prefix}{out}_{metric}" if prefix else f"{out}_{metric}"
                    value = logs.get(key, None)
                    row.append(f"{value:.4f}" if value is not None else "-")
                rows.append(row)
        print(tabulate(rows, headers=headers, tablefmt="fancy_grid"))

# 4. Load and Preprocess Data

In [5]:
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath)

    df = df.dropna(subset=["Lambda", "Hinge B", "Hinge C"])
    df = df[df["Lambda"] != 0.0]

    X = df[["Width", "Span", "Thickness", "Unit Weight", "Blocks", "Load Position"]].values
    y = df[["Lambda", "Hinge B", "Hinge C"]].values

    X[:, 3] = X[:, 3] / 100  # Normalize Unit Weight

    X_std = StandardScaler().fit_transform(X)
    X_scaled = MinMaxScaler().fit_transform(X_std)

    y_scalers = [MinMaxScaler() for _ in range(y.shape[1])]
    y_scaled = np.column_stack([
        scaler.fit_transform(y[:, i].reshape(-1, 1)).flatten()
        for i, scaler in enumerate(y_scalers)
    ])

    X_temp, X_test, y_temp, y_test = train_test_split(X_scaled, y_scaled, test_size=0.3, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=2/7, random_state=42)

    return X_train, y_train, X_val, y_val, X_test, y_test, y_scalers

def split_targets(y):
    return {
        'Collapse_factor': y[:, 0],
        'Hinge_1': y[:, 1],
        'Hinge_2': y[:, 2],
    }


# 5. Build the ANN Model


In [6]:
def ANN(input_dim):
    inputs = Input(shape=(input_dim,))
    x = Dense(256, activation='relu')(inputs)
    x = Dropout(0.2)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(16, activation='relu')(x)

    out_collapse = Dense(1, name='Collapse_factor')(x)
    out_hingeB = Dense(1, name='Hinge_1')(x)
    out_hingeC = Dense(1, name='Hinge_2')(x)

    model = Model(inputs=inputs, outputs={
        'Collapse_factor': out_collapse,
        'Hinge_1': out_hingeB,
        'Hinge_2': out_hingeC
    })

    model.compile(
        optimizer='adam',
        loss='mse',
        metrics={
            'Collapse_factor': [MeanSquaredError(name='mse'), MeanAbsoluteError(name='mae'), R2Score(name='r2_score')],
            'Hinge_1': [MeanSquaredError(name='mse'), MeanAbsoluteError(name='mae'), R2Score(name='r2_score')],
            'Hinge_2': [MeanSquaredError(name='mse'), MeanAbsoluteError(name='mae'), R2Score(name='r2_score')],
        }
    )
    return model


# 6. Load Data & Train the Model

In [8]:
def lr_schedule(epoch):
    initial_lr = 0.001
    drop = 0.5
    epoch_drop = 10
    return initial_lr * (drop ** (epoch // epoch_drop))

csv_path = "arch_dataset_outputs.csv"
X_train, y_train, X_val, y_val, X_test, y_test, y_scalers = load_and_preprocess_data(csv_path)
y_train_dict = split_targets(y_train)
y_val_dict = split_targets(y_val)
y_test_dict = split_targets(y_test)

ann_model = ANN(X_train.shape[1])
lr_scheduler = LearningRateScheduler(lr_schedule)

print("✅ Starting training...\n")

ann_model.fit(
    X_train, y_train_dict,
    validation_data=(X_val, y_val_dict),
    epochs=300,
    batch_size=64,
    callbacks=[lr_scheduler, EpochLogger()],
    verbose=0
)

✅ Starting training...



/home/aditi/.local/lib/python3.10/site-packages/keras/src/engine/training.py:2723: UserWarning: Metric R2Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()



📌 Epoch 001

╒═════════════════════╤════════╤════════╤═════════╕
│ Output              │    MSE │    MAE │      R² │
╞═════════════════════╪════════╪════════╪═════════╡
│ Collapse_factor     │ 0.0004 │ 0.009  │ -5.1158 │
├─────────────────────┼────────┼────────┼─────────┤
│ Hinge_1             │ 0.0075 │ 0.0616 │  0.8712 │
├─────────────────────┼────────┼────────┼─────────┤
│ Hinge_2             │ 0.0133 │ 0.0871 │  0.7495 │
├─────────────────────┼────────┼────────┼─────────┤
│ val_Collapse_factor │ 0.0001 │ 0.0032 │ -0.1585 │
├─────────────────────┼────────┼────────┼─────────┤
│ val_Hinge_1         │ 0.0031 │ 0.0413 │  0.9468 │
├─────────────────────┼────────┼────────┼─────────┤
│ val_Hinge_2         │ 0.0062 │ 0.0561 │  0.883  │
╘═════════════════════╧════════╧════════╧═════════╛

📌 Epoch 002

╒═════════════════════╤════════╤════════╤═════════╕
│ Output              │    MSE │    MAE │      R² │
╞═════════════════════╪════════╪════════╪═════════╡
│ Collapse_factor     │ 0.0001 │ 0.0

# 7. Evaluate on Test Set

In [9]:
print("\n📊 Final Evaluation on Test Set:\n")
test_preds = ann_model.predict(X_test)
true = y_test
pred = np.column_stack([
    test_preds['Collapse_factor'].flatten(),
    test_preds['Hinge_1'].flatten(),
    test_preds['Hinge_2'].flatten()
])

r2_scores = r2_score(true, pred, multioutput='raw_values')
mse = tf.keras.losses.MeanSquaredError()
mae = tf.keras.losses.MeanAbsoluteError()

headers = ["Output", "MSE", "MAE", "R²"]
rows = []
output_keys = ['Collapse_factor', 'Hinge_1', 'Hinge_2']
for i, key in enumerate(output_keys):
    rows.append([
        key,
        f"{mse(true[:, i], pred[:, i]).numpy():.4f}",
        f"{mae(true[:, i], pred[:, i]).numpy():.4f}",
        f"{r2_scores[i]:.4f}"
    ])

print(tabulate(rows, headers=headers, tablefmt="fancy_grid"))


📊 Final Evaluation on Test Set:

4023/4023 [==============================] - 6s 2ms/step
╒═════════════════╤════════╤════════╤════════╕
│ Output          │    MSE │    MAE │     R² │
╞═════════════════╪════════╪════════╪════════╡
│ Collapse_factor │ 0.0001 │ 0.0014 │ 0.0856 │
├─────────────────┼────────┼────────┼────────┤
│ Hinge_1         │ 0.0007 │ 0.0181 │ 0.9889 │
├─────────────────┼────────┼────────┼────────┤
│ Hinge_2         │ 0.0011 │ 0.0226 │ 0.98   │
╘═════════════════╧════════╧════════╧════════╛


# 8. Sample Predictions

In [10]:
y_test_original = np.column_stack([
    y_scalers[i].inverse_transform(y_test[:, i].reshape(-1, 1)).flatten()
    for i in range(y_test.shape[1])
])

pred_original = np.column_stack([
    y_scalers[i].inverse_transform(test_preds[key].reshape(-1, 1)).flatten()
    for i, key in enumerate(output_keys)
])

rng = np.random.default_rng()
sample_indices = rng.choice(len(X_test), size=5, replace=False)

print("\n📌 Sample Predictions (True vs Predicted):\n")

headers = ["Sample", 
           "Collapse (True)", "Collapse (Pred)", 
           "Hinge 1 (True)", "Hinge 1 (Pred)", 
           "Hinge 2 (True)", "Hinge 2 (Pred)"]

rows = []
for i, idx in enumerate(sample_indices):
    row = [f"Sample {i+1}"]
    for j in range(len(output_keys)):
        row.append(round(y_test_original[idx, j], 4))     
        row.append(round(pred_original[idx, j], 4))       
    rows.append(row)

print(tabulate(rows, headers=headers, tablefmt="fancy_grid"))



📌 Sample Predictions (True vs Predicted):

╒══════════╤═══════════════════╤═══════════════════╤══════════════════╤══════════════════╤══════════════════╤══════════════════╕
│ Sample   │   Collapse (True) │   Collapse (Pred) │   Hinge 1 (True) │   Hinge 1 (Pred) │   Hinge 2 (True) │   Hinge 2 (Pred) │
╞══════════╪═══════════════════╪═══════════════════╪══════════════════╪══════════════════╪══════════════════╪══════════════════╡
│ Sample 1 │            3.0812 │          -24.1591 │               18 │          18.9888 │               23 │          24.5051 │
├──────────┼───────────────────┼───────────────────┼──────────────────┼──────────────────┼──────────────────┼──────────────────┤
│ Sample 2 │           10.6028 │           57.0055 │               25 │          25.0565 │               39 │          38.9932 │
├──────────┼───────────────────┼───────────────────┼──────────────────┼──────────────────┼──────────────────┼──────────────────┤
│ Sample 3 │            0.4158 │            3.1094 │ 